<a href="https://colab.research.google.com/github/motorio0829/DL-for-AI/blob/main/NLP%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EC%8A%A4%ED%83%80%EC%9D%BC_%ED%8C%90%EB%B3%84%EA%B8%B0%2C_%EB%B3%80%ED%99%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import transformers, sys, platform, torch
print(transformers.__version__)   # 4.x 가 아니라면 업그레이드 실패

4.51.3


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd, datasets as ds
target = {"formal":0, "banmal":1}         # 원하는 두 스타일만 지정

df = pd.read_csv("/content/drive/MyDrive/nlp_project/smilestyle_dataset.tsv", sep="\t")

In [ ]:
df.head(2)

,formal,informal,android,azae,chat,choding,emoticon,enfp,gentle,halbae,halmae,joongding,king,naruto,seonbi,sosim,translator
0,안녕하세요. 저는 고양이 6마리 키워요.,안녕! 나는 고양이 6마리 키워.,휴먼. 반갑다. 안드로이드는. 고양이. 6마리. 소유중.,아이고 안녕하십니까~ 나는 그냥 고양이 6마리 키우고 있는 사람이여,하잉ㅋㅋ 나 떼걸룩 6마리 키운다!,ㅎㅇ 나 주인님 6마리 모심 ㅋㅋ,안녕!! >< 나는 고양이😺를 ➏ 마리 키우고있어!! 0_0,안녕안녕~! 나 고양이 6마리나 키운다? 완전 대박이징~,"안녕하십니까,, 저는 고양이 6마리 키웁니다.",안녕하신가~... 난 지금 고양이를 6마리 키우고 있다네,하유 시벌것 괭이놈 6마리 키우는데 힘들어 죽겟네,안녕하냐 ㅡㅡ 나 씹냥이 6마리나 키운다 하;,반갑소. 짐은 고양이를 6마리나 키우오.,안녕하냐니깐! 난 고양이를 6마리 키우고있다니깐!,안녕하시오! 소인은 고양이를 6마리 키우고 있소!,안녕… 난 고양이 6마리 키워 ㅠㅠ,반가운. 나는 6마리의 고양이를 소지하고 있다.
1,고양이를 6마리나요? 키우는거 안 힘드세요?,고양이를 6마리나? 키우는거 안 힘들어?,고양이. 6마리. 양육. 번거로운가.,아니 무슨 고양이를 6마리나? 거 키우는 거 안 힘든가?,엥? 6마리나? 안힘듬?ㅋㅋㅋㅋ,6마리? 에바아니냐 안 힘듦?,고양이를 6마리나?!! w(ﾟДﾟ)w 키우는거 안 힘듬?? (⊙_⊙;),고양이를 6마리나? 완전 대박~ 키우는 거 안 힘들어?!,"고양이를 6마리나 키우십니까? 안 힘드신지,,",고양이를 6마리나? 키우는거 힘들지 않는가?,니기럴 털만 날리는 거 키우기 안 힘들데?,아니 고양이를 6마리나? 안힘드냐?,고양이를 6마리나? 키우는게 수고스럽진 않소?,고양이를 6마리나? 키우는거 힘들지 않냐니깐?,고양이를 6마리나 키우고 있는 것이오? 힘들지 않소?,고양이..6마리나? ㅠ 키우는건 혹시 안힘들어..?,6마리의 고양이? 당신은 그들로부터 지치지 않습니까?


## 판별

In [ ]:
# ② 두 열만 선택
df2 = df[["formal", "informal"]].copy()          # 'informal' = 반말

# ③ melt 로 세로로 쌓기
long_df = df2.melt(value_vars=["formal", "informal"],
                   var_name="style", value_name="text").dropna()

# ④ 라벨 부여
target = {"formal": 0, "informal": 1}
long_df["label"] = long_df["style"].map(target)

long_df.head()

,style,text,label
0,formal,안녕하세요. 저는 고양이 6마리 키워요.,0
1,formal,고양이를 6마리나요? 키우는거 안 힘드세요?,0
2,formal,제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.,0
3,formal,가장 나이가 많은 고양이가 어떻게 돼요?,0
4,formal,여섯 살입니다. 갈색 고양이에요.,0


In [ ]:
import datasets as ds
from datasets import ClassLabel
dset = ds.Dataset.from_pandas(long_df[["text", "label"]])
dset = dset.cast_column("label", ClassLabel(num_classes=2, names=["formal", "informal"]))

dset = dset.train_test_split(
    test_size=0.1,
    stratify_by_column="label",   # 라벨 균형 유지
    seed=42
)
print(len(dset["train"]), len(dset["test"]))

Casting the dataset:   0%|          | 0/6940 [00:00<?, ? examples/s]

6246 694


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from evaluate import load as load_metric

tok = AutoTokenizer.from_pretrained("klue/roberta-base")
def tokenize(batch):
    return tok(batch["text"], truncation=True, padding="max_length",
               max_length=128)
d_tok = dset.map(tokenize, batched=True).rename_column("label","labels")
d_tok.set_format("torch")

model = AutoModelForSequenceClassification.from_pretrained(
            "klue/roberta-base", num_labels=2)

from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir="clf_ckpt",
    per_device_train_batch_size=32,
    num_train_epochs=3,
    eval_strategy="epoch",   # ← eval→evaluation
    save_strategy="epoch",         # (load_best_model_at_end 쓰려면 일치)
    load_best_model_at_end=True,
    fp16=True,
    metric_for_best_model="f1",
    report_to="none",              # wandb 끄기
)

from evaluate import load as load_metric

metric_acc = load_metric("accuracy")
metric_f1  = load_metric("f1")

# 2) Trainer에 넘길 metrics 함수
def metrics(p):
    preds = p.predictions.argmax(-1)
    refs  = p.label_ids
    return {
        "acc": metric_acc.compute(predictions=preds, references=refs)["accuracy"],
        "f1" : metric_f1.compute(predictions=preds, references=refs, average="macro")["f1"],
    }

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=d_tok["train"],  # ← 키워드로 명시
    eval_dataset=d_tok["test"],
    compute_metrics=metrics,
)

trainer.train()
model.save_pretrained("clf_ckpt")
tok.save_pretrained("clf_ckpt")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Acc,F1
1,No log,0.016221,0.997118,0.997118
2,No log,0.028652,0.995677,0.995677
3,0.044700,0.013038,0.997118,0.997118


('clf_ckpt/tokenizer_config.json',
 'clf_ckpt/special_tokens_map.json',
 'clf_ckpt/vocab.txt',
 'clf_ckpt/added_tokens.json',
 'clf_ckpt/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

clf_dir = "clf_ckpt"
tok = AutoTokenizer.from_pretrained(clf_dir)
model = AutoModelForSequenceClassification.from_pretrained(clf_dir).eval().cuda()

def predict_style(text):
    inputs = tok(text, return_tensors="pt", truncation=True,
                 padding=True, max_length=128).to("cuda")
    with torch.no_grad():
        probs = torch.softmax(model(**inputs).logits, -1)[0]
    return "formal" if probs[0] > probs[1] else "informal", probs.cpu().tolist()

In [ ]:
print(predict_style("야 뭐하냐?"))

('informal', [0.0011402162490412593, 0.9988597631454468])


In [ ]:
print(predict_style('안녕하세요 반갑습니다'))

('formal', [0.9908374547958374, 0.009162629023194313])


In [ ]:
print(predict_style('오늘 날씨는 맑습니다'))

('formal', [0.9910823106765747, 0.008917690254747868])


## 변환

In [ ]:
import pandas as pd, datasets as ds
RAW = "/content/drive/MyDrive/nlp_project/smilestyle_dataset.tsv"         # 파일 위치

df = pd.read_csv(RAW, sep="\t")[["formal", "informal"]].dropna()
df.reset_index(inplace = True)
df

,index,formal,informal
0,0,안녕하세요. 저는 고양이 6마리 키워요.,안녕! 나는 고양이 6마리 키워.
1,1,고양이를 6마리나요? 키우는거 안 힘드세요?,고양이를 6마리나? 키우는거 안 힘들어?
2,2,제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.,내가 워낙 고양이를 좋아해서 크게 힘들진 않아.
3,3,가장 나이가 많은 고양이가 어떻게 돼요?,가장 나이가 많은 고양이가 몇 살이야?
4,4,여섯 살입니다. 갈색 고양이에요.,여섯 살이야. 갈색 고양이지.
...,...,...,...
3465,3700,"블랙 미러도 재미있다고 들었는데, 어떤가요?","블랙 미러도 재미있다고 들었는데, 어때?"
3466,3701,저는 티비를 자주 안 봐서 사실 잘 몰라요.,나는 티비를 자주 안 봐서 사실 잘 몰라.
3467,3702,그러면 직접 알아봐야겠네요.,그러면 직접 알아봐야겠네.
3468,3703,새로운 티비쇼를 개척하는 것도 재밌을거에요.,새로운 티비쇼를 개척하는 것도 재밌을거야.


In [ ]:
long_df2 = df.melt(id_vars="index",
               value_vars=["formal", "informal"],
               var_name="style", value_name="text")

# ③ row_id 기준 pivot
pairs = (long_df2.pivot(index="index", columns="style", values="text")
              .dropna(subset=["formal", "informal"])
              .reset_index(drop=True))

print(pairs.head(), len(pairs))
pairs.to_csv("pairs.tsv", sep="\t", index=False)

style                       formal                    informal
0           안녕하세요. 저는 고양이 6마리 키워요.          안녕! 나는 고양이 6마리 키워.
1         고양이를 6마리나요? 키우는거 안 힘드세요?      고양이를 6마리나? 키우는거 안 힘들어?
2      제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.  내가 워낙 고양이를 좋아해서 크게 힘들진 않아.
3           가장 나이가 많은 고양이가 어떻게 돼요?       가장 나이가 많은 고양이가 몇 살이야?
4               여섯 살입니다. 갈색 고양이에요.            여섯 살이야. 갈색 고양이지. 3470


In [ ]:
import datasets as ds
from transformers import AutoTokenizer

# ① TSV → Dataset
dset = ds.load_dataset("csv", data_files="pairs.tsv",
                       delimiter="\t")["train"]

# ② train/validation 분할 (9:1)
split = dset.train_test_split(test_size=0.1, seed=42)

# ③ KoBART 토크나이저
MAX_LEN = 96
tok = AutoTokenizer.from_pretrained("gogamza/kobart-base-v2", use_fast=True)

def tok_fn(batch):
    model_in  = tok(batch["formal"],  truncation=True, padding="max_length", max_length=MAX_LEN)
    model_out = tok(batch["informal"], truncation=True, padding="max_length", max_length=MAX_LEN)
    model_in["labels"] = model_out["input_ids"]
    return model_in

ds_tok = split.map(tok_fn, batched=True, remove_columns=split["train"].column_names)
ds_tok.set_format("torch")
print(len(ds_tok["train"]), len(ds_tok["test"]))   # → 3123  347

Generating train split: 0 examples [00:00, ? examples/s]

config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.


tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.


Map:   0%|          | 0/3123 [00:00<?, ? examples/s]

Map:   0%|          | 0/347 [00:00<?, ? examples/s]

3123 347


In [ ]:
from transformers import (
    BartForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

CHECKPOINT = "gogamza/kobart-base-v2"   # 또는 skt/kobart-base
model = BartForConditionalGeneration.from_pretrained(CHECKPOINT)

args = Seq2SeqTrainingArguments(
    output_dir="kobart_style",
    overwrite_output_dir=True,
    num_train_epochs=5,                 # 데이터 3k → 4~5 epoch 권장
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    eval_strategy="epoch",
    save_strategy="epoch",
    predict_with_generate=True,
    fp16=True,
    logging_steps=50,
    report_to="none",                   # wandb 끄기
)

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=ds_tok["train"],
    eval_dataset=ds_tok["test"],
    tokenizer=tok,
    data_collator=DataCollatorForSeq2Seq(tok, model),
)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.


model.safetensors:   0%|          | 0.00/495M [00:00<?, ?B/s]

<ipython-input-17-4b13957994e3>:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

# 완료 후 저장
model.save_pretrained("kobart_style")
tok.save_pretrained("kobart_style")

/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:741: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Epoch,Training Loss,Validation Loss
1,0.041500,0.033050
2,0.024200,0.031679
3,0.008000,0.033643
4,0.006600,0.033663
5,0.002300,0.035122


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'forced_eos_token_id': 1}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('kobart_style/tokenizer_config.json',
 'kobart_style/special_tokens_map.json',
 'kobart_style/tokenizer.json')

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

model = BartForConditionalGeneration.from_pretrained("kobart_style").to(device)
tok   = AutoTokenizer.from_pretrained("kobart_style")
MAX_LEN = 96

def convert(text, beams=5):
    ids = tok(text,
              return_tensors="pt",
              truncation=True, max_length=MAX_LEN).to(device)

    # ▶ token_type_ids 제거
    ids.pop("token_type_ids", None)

    with torch.no_grad():
        out = model.generate(**ids,
                             num_beams=beams,
                             max_new_tokens=MAX_LEN)
    return tok.decode(out[0], skip_special_tokens=True)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.


In [ ]:
# 간단 테스트
tests = [
    "네이버와 다음이 제21대 대통령 선거를 앞두고 대선 특집 페이지를 마련했다.",
    "대선 관련 기사, 여론조사, 투표소 정보 등을 주요 정보를 제공한다",
    "여론 조작을 막기 위해 댓글 조작과 같은 매크로와 허위조작정보 전달 등 부정행위 단속을 강화한다",
    "안녕하십니까. 반갑습니다"
]
for s in tests:
    print("▶", convert(s))

▶ 네이버와 다음이 제21대 대통령 선거를 앞두고 대선 특집 페이지를 마련했어.
▶ 대선 관련 기사, 여론조사, 투표소 정보 등을 주요 정보를 제공한다 .
▶ 여론 조작을 막기 위해 댓글 조작과 같은 매크로와 허위조작정보 전달 등 부정행위 단속을 강화한다 .
▶ 안녕. 반갑네.


In [ ]:
import json, pandas as pd

path = "/content/drive/MyDrive/nlp_project/NIKL_NEWSPAPER_2023_v1.0/NIRW2300000001.json"

with open(path, encoding="utf-8") as f:
    data = json.load(f)

In [ ]:
sentence_list = []
for i in range(0, len(data['document']), 100):
    for j in range(1, len(data['document'][i]['paragraph'])):
        sentence_list.append(data['document'][i]['paragraph'][j]['form'])

In [ ]:
import pandas as pd, re

def sent_split(text):
    s = re.split(r"(?<=[.!?])\s+", text.strip())
    return [x for x in s if 15 < len(x) < 120]

sentences = []
for body in sentence_list:
    sentences.extend(sent_split(body))
sentences = list(dict.fromkeys(sentences))
print("문장 수:", len(sentences))

문장 수: 6201


In [ ]:
sentences

['취임 2년 차를 맞는 박형준 부산시장은 코로나19 위기를 이겨내고 있는 부산시민들에게 감사의 인사를 먼저 전했다.',
 '박 시장은 “코로나19는 번번이 우리의 희망의 길목을 막아섰고, 지금도 여전히 민생을 위협하고 있다”며 “그러나, 부산시민들께서는 위기 때마다 힘을 모아주셨다”고 말했다.',
 '그러면서 “숱한 어려움을 헤쳐온 위대한 시민의 힘으로 2022년을 코로나를 극복하고 일상을 회복하는 해, ‘그린 스마트 도시 부산’을 향한 대도약의 원념으로 삼고자 한다”고 새해 포부를 밝혔다.',
 '박 시장은 “치밀한 전략으로 2030세계박람회 유치를 위한 국제박람회기구 현지 실사를 성공적으로 이끌고 시민의 염원과 기대를 온전히 담아 북항재개발사업과 가덕도신공항 사업을 본궤도에 올리겠다”고 약속했다.',
 '또, “부·울·경 특별자치단체 출범과 함께 수도권에 대응할 초광역 협력체계를 구축해 기후위기 대응과 탄소중립, 그리고 이를 구현할 디지털 전환을 어느 도시보다 빠르게 준비하겠다”고 다짐했다.',
 '박 시장은 “완전히 새로워지는 부산, 그 안에서 행복한 일상을 누리는 부산시민의 모습을 그려본다”며 “그 초석을 다질 2022년, 새 희망을 함께 만들어나가자”고 말했다.',
 '신상해 부산시의회 의장은 ‘따뜻한 회복, 중단없는 발전’을 위해 계속 전진해나가겠다고 다짐했다.',
 '신 의장은 “돌이켜보면 지난 한 해 부산은 고군분투하며 위기 속에 기회를 만드는 저력을 발휘했다”며 “모두가 시민 여러분 덕분”이라고 감사의 뜻을 전했다.',
 '이어 “새해를 맞는 부산시의회 각오는 변함없이 결연하다”며 “코로나19 직격탄을 맞은 자영업·소상공인 그리고 취약계층이 하루빨리 일어설 수 있도록 ‘따뜻한 회복’에 매진하겠다”고 말했다.',
 '신 의장은 “오래 염원해온 미래를 하루라도 빨리 앞당기기 위해 우리는 코로나19와의 싸움과 수도권 일극체제와의 대결에서 반드시 반전의 기회를 잡아야 한다”며 “힘을 모아 기필코 승리하자”고 강조했다.',
 '김석준 부산시교육감 역시

In [ ]:
# 0) 의존 패키지
!pip install -q openai pandas tqdm

import os, time, pandas as pd
from tqdm.auto import tqdm
from openai import OpenAI, OpenAIError

#os.environ["OPENAI_API_KEY"] = ""

# 2) 원본 CSV 읽기
df = pd.read_csv("sentences.csv", encoding="utf-8-sig")   # ↔ cp949 등 필요 시 수정
assert "formal" in df.columns, "CSV에 'formal' 열이 없습니다."

In [ ]:
# 3) LLM 호출 설정
client  = OpenAI()
# 프롬프트 수정하기 좀 더 명확하게
PROMPT  = ("다음 문장을 **반말**과 **쉬운 단어**로 자연스럽게 바꿔줘. "
           "문장 하나로만 출력해 줘.\n\n\"{sent}\"\n\n###")

def to_informal(src: str, retry: int = 3) -> str:
    for _ in range(retry):
        try:
            rsp = client.chat.completions.create(
                model="gpt-4o-mini", # 모델 설정(싼걸로)
                messages=[{"role": "user",
                           "content": PROMPT.format(sent=src)}],
                temperature=0.7,
                max_tokens=128,
            )
            return rsp.choices[0].message.content.strip()
        except OpenAIError as e:    # RateLimit 등 재시도
            time.sleep(1.5)
    return src    # 실패 시 원문 유지

# 4)  변환 실행 (200~5000 문장도 무료 크레딧 안)
informals = [to_informal(s) for s in tqdm(df["formal"], desc="Converting")]
df["informal"] = informals

# 5)  결과 저장 (엑셀 호환)
out_name = "sentences_informal.csv"
df.to_csv(out_name, index=False, encoding="utf-8-sig")
print(f"저장 완료 → {out_name}")